In [0]:
# Amit Kumar
# Nikhilesh
# Shivam

In [0]:
!python prepare_data.py shakespeare.txt skp

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from prepare_data import parse_seq
import pickle
import tensorflow as tf

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x, 200))

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

In [0]:
# create weights variables
def make_weights(i, o, initializer):
    return tf.Variable(initializer(shape=[i, o], dtype=tf.float32))

# create bias variables
def make_bias(k, initializer):
    return tf.Variable(initializer(shape=[k], dtype=tf.float32))

In [0]:
# RNN model

# tf.Variable(
#     initial_value=None, trainable=None, validate_shape=True, caching_device=None,
#     name=None, variable_def=None, dtype=None, import_scope=None, constraint=None,
#     synchronization=tf.VariableSynchronization.AUTO,
#     aggregation=tf.compat.v1.VariableAggregation.NONE, shape=None
# )

w_range = 0.1
i_layer = vocab_size
h_layer = 512
o_layer = vocab_size
batch_size = 128
batch_seq_length = 200

# data = data.shuffle(60000)
data = data.batch(batch_size)
# data = data.repeat()

w_xh = make_weights(i_layer, h_layer, tf.initializers.RandomUniform(minval=-w_range, maxval=w_range))
w_hh = make_weights(h_layer, h_layer, tf.initializers.RandomUniform(minval=-w_range, maxval=w_range))

b_h = make_bias(h_layer, tf.initializers.constant(0.001))

w_ho = make_weights(h_layer, o_layer, tf.initializers.RandomUniform(minval=-w_range, maxval=w_range))

b_o = make_bias(o_layer, tf.initializers.constant(0.001))


opt = tf.optimizers.Adam()
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [0]:
data

<BatchDataset shapes: (None, 200), types: tf.int32>

In [0]:
# print('w_xh : {}\n'.format(w_xh))
# print('w_hh : {}\n'.format(w_hh))
# print('b_h : {}\n'.format(b_h))
# print('w_ho : {}\n'.format(w_ho))
# print('b_o : {}\n'.format(b_o))

In [0]:
def one_hot_conversion(data, vocab_size):
    return tf.one_hot(indices=data,depth=vocab_size);

In [0]:
# some more model :(

def rrn_model(inp, h):
    h = tf.nn.tanh(tf.matmul(h, w_hh) + tf.matmul(inp, w_xh) + b_h)
    o = tf.nn.softmax(tf.matmul(h, w_ho) + b_o)
    return h, o

In [0]:
# tf.constant(
#     value, dtype=None, shape=None, name='Const'
# )

train_steps = 2000

# @tf.function
# def very_difficult_function(batch_size, batch_seq_length):
for step, batch_seq in enumerate(data):
#     print('batch_seq : {}'.format(batch_seq))
    with tf.GradientTape() as tape:
        
        loss = tf.constant(0, dtype=tf.float32, shape=None)
#         print('Initial Loss : {}'.format(loss))
        h = tf.zeros([batch_size, h_layer])
#         print('Initial Hidden State : {}'.format(h))
        
        for seq in tf.range(batch_seq_length - 1):
#             print('Sequence : {}'.format(seq))
            
            train_data = batch_seq[:, seq]
            # next character as ground truth
            ground_truth = batch_seq[:, seq+1] 
            inp = one_hot_conversion(train_data, vocab_size)
            h, o = rrn_model(inp, h)
            xent = loss_fn(ground_truth, o)
            loss = loss + xent
        loss = loss / (batch_seq_length -1)
#         print('Loss : {}'.format(loss))
    varis = [w_xh, w_hh, b_h, w_ho, b_o]
    grads = tape.gradient(xent, varis)
    opt.apply_gradients(zip(grads, varis))
    if not step % 100:
        print("Step: {} Loss: {}\n".format(step, xent))
        
    if step > train_steps:
        break
    

In [0]:
train_steps = 2000

for step, batch_seq in enumerate(data):
    
    if step > train_steps:
        break
    
    xent = very_difficult_function(batch_size, batch_seq)
    
    
    if not step % 100:
        print("Step: {} Loss: {}\n".format(step, xent))

